In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [568]:
class NeuralNetwork(object):
    
    def __init__(self):
        self.layer_count = 1
        self.parameters = {}
        self.results = {}
        
    def W_initializer(self, shape):
        
        '''
        This function will return a matrix of normally distributed random values of 0 mean and 1 std. 
        Parameters:
        shape: List, Tuple
            Shape will be a tuple of two values input shape and output shape values. 
        '''
        return np.random.randn(shape[0], shape[1])
    
    def Bias_initializer(self, shape):
        '''
        This function will return a vector of normally distributed random values of 0 mean and 1 std. 
        parameters
        shape: int
            Output Vector size
        '''
        return np.random.normal(size = shape)
    
    def sigmoid(self, x):
        '''
        This function will return sigmoid activated value. 
        parameters
        x:
            x will be the power value, a matrix multiplication of theta vector and feature vector. 
        '''
        return 1 / (1 * np.exp(- x))
    
    def relu(self, x):
        '''
        ReLU is an activation function
        '''
        return np.maximum(x, np.zeros(x.shape))
    
    def soft_max(self, x):
        
        '''
        Soft max is an activation function vector. 
        parameters
        x: np.array
            x will be the power values
        '''
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis = 1)[:, None]
    
    def add_layer(self, in_shape, out_shape, activation = 'relu'):
        
        '''
        This funtion will add a layer into the neural network. Assigning all the parameters to the neural network. 
        Also assigning the activation function into the paramters. 
        '''
        
        self.parameters['layer: ' + str(self.layer_count)] = {'W': self.W_initializer((in_shape, out_shape)), 'b': np.ones((out_shape))}
        
        if (activation == 'relu'):
            self.parameters['layer: ' + str(self.layer_count)]['Activation'] = self.relu
        elif (activation == 'sigmoid'):
            self.parameters['layer: ' + str(self.layer_count)]['Activation'] = self.sigmoid
        elif (activation == 'soft_max'):
            self.parameters['layer: ' + str(self.layer_count)]['Activation'] = self.soft_max
            
        self.layer_count += 1
    
    def dense_layer(self, x, parameters):
        
        '''
        Dense Layer is the matrix multiplication of Weight matrix and feature vector then adding the bias in it. 
        Activation function will call after operation. 
        '''
        W = parameters['W']
        bias = parameters['b']
        activation = parameters['Activation']
        Z = np.matmul(x, W) + bias
        return activation(Z)
    
    
    def forward_pass(self, x):
        
        '''
        Forward Pass
        '''
        result = x
        self.results['A0'] = result
        for i in range(1, self.layer_count):
            result = self.dense_layer(result, self.parameters['layer: ' + str(i)])
            self.results['A' + str(i)] = result
        return result
    
    
    def del_forward_pass(self):
        '''
        This function will calculate the derivatives for the forward pass. 
        '''
        
        dev_forpass = {}
        for i in range(1, self.layer_count):
            act = self.parameters['layer: ' + str(i)]['Activation'] ## Taking activation for calculating different derivatives
            result = self.results['A' + str(i - 1)]
            if (act == self.relu):
                dev_forpass['layer: ' + str(i)] = {'dW': result.T, 'db': result.T}
            elif (act == self.sigmoid):
                sig = self.sigmoid(result.T)
                dev_forpass['layer: ' + str(i)] = {'dW': ((sig * (1 - sig)) * result.T), 'db': (1 - sig)}

        return dev_forpass
    
    def del_backward_pass(self, in_shape, error):
        
        '''
        In backward propogation we have to calculate all the values 
        
        '''
        dev_backpass = {}
        theta_mul = np.ones((in_shape, 1)) * error ## Last Layer dL / dA
        
        for i in range(self.layer_count - 1, 1, -1):

            dev_backpass['layer: '+ str(i)] = {'dW': theta_mul, 'db': 1} ## Storing del Activations 
            theta_val = self.parameters['layer: ' + str(i)] ## Taking thetas of l + 1 layer
            theta_mul =  theta_mul @ theta_val['W'].T ## Calculating del Activation 

        dev_backpass['layer: '+ str(i - 1)] = {'dW': theta_mul, 'db': 1}
        return dev_backpass
    
    
    def predict_prob(self, x):
        return self.forward_pass(x)
    
    def predict(self, x):
        return np.argmax(self.forward_pass(x))
    
    def GradientDescentOptimizer(self, x, y, alpha, epoch = 100):
        loss_dir = []
        for _ in range(epoch):
            
            result = self.forward_pass(x)
            del_loss = self.biclass_cross_entropy(x, y)
            del_forward = self.del_forward_pass()
            del_backward = self.del_backward_pass(x.shape[0], del_loss)
            
            for i in range(1, self.layer_count):
                self.parameters['layer: ' + str(i)]['W'] -= alpha * (del_forward['layer: ' + str(i)]['dW'] @ del_backward['layer: ' + str(i)]['dW'])
#                 self.parameters['layer: ' + str(i)]['b'] -= alpha * (del_forward['layer:' + str(i)]['db'] @ del_backward['layer: ' + str(i)]['db'])

            loss = self.Biclass_Loss(result, y)
            print ('Loss: ', loss)
            loss_dir.append(loss)
        return loss_dir
    
    def AdamOptimizer(self, x, y, alpha, epoch = 100):
        pass
    
    def AdagradeOptimizer(self, x, y, alpha, epoch = 100):
        pass
        
    def RMSpropOptimizer(self, x, y, alpha, epoch = 100):
        pass
    
    def MomentumUpdateOptimizer(self, x, y, alpha, epoch = 100):
        pass
    
    def NesterovMomentumOptimizer(self, x, y, alpha, epoch = 100):
        pass
    
    def softmax_cross_entropy(self, logits, labels):
        return (- np.mean( labels - np.log(self.soft_max(logits))))
    
    def Biclass_Loss(self, x, y):
        sig = self.sigmoid(x)
        return -np.mean( (y * np.log(sig)) + ((1 - y) * np.log(1 - sig)))
    
    def multiclass_Loss(self, x, y):
        return -np.mean(y * np.log(self.soft_max(x)))
    
    def biclass_cross_entropy(self, x, y):
        return -np.mean((np.log(self.sigmoid(x)) - y) * x)
    
    def fit(self, x, y, learning_rate, optimizer = 'grad_dst', epoch = 100):
        
        if (optimizer == 'grad_dst'):
            self.GradientDescentOptimizer(x, y, learning_rate, epoch)
        elif (optimizer == 'adam'):
            pass
        elif (optimizer == 'rmsprop'):
            pass
        elif(optimizer == 'adagrade'):
            pass

In [575]:
nn = NeuralNetwork()
nn.add_layer(3, 3, 'sigmoid')
nn.add_layer(3, 2, 'sigmoid')
nn.add_layer(2, 1, 'sigmoid')

In [576]:
nn.parameters

{'layer: 1': {'W': array([[-0.20264511,  1.45125769,  1.50001282],
         [ 0.55321011,  0.80195992,  0.00647126],
         [ 0.87300533,  0.94163917, -0.837597  ]]),
  'b': array([1., 1., 1.]),
  'Activation': <bound method NeuralNetwork.sigmoid of <__main__.NeuralNetwork object at 0x0000027638A34550>>},
 'layer: 2': {'W': array([[-0.75136743,  0.78957496],
         [-1.9245775 , -0.93031235],
         [-0.81521169,  0.40858933]]),
  'b': array([1., 1.]),
  'Activation': <bound method NeuralNetwork.sigmoid of <__main__.NeuralNetwork object at 0x0000027638A34550>>},
 'layer: 3': {'W': array([[ 1.29944781],
         [-0.09488551]]),
  'b': array([1.]),
  'Activation': <bound method NeuralNetwork.sigmoid of <__main__.NeuralNetwork object at 0x0000027638A34550>>}}

In [571]:
data = np.random.normal(5, 2, (128, 3))
y = np.random.randint(0, 2, (128, 1))

In [573]:
nn.fit(data, y, 0.01, epoch = 1)

Loss:  nan


In [542]:
forward = nn.del_forward_pass()

In [543]:
backwar = nn.del_backward_pass(128, 2)

In [544]:
del_layer = {}

for i in range(1, nn.layer_count):
    del_layer['layer: ' + str(i)] = forward['layer: ' + str(i)]['dW'] @ backwar['layer: ' + str(i)]['dW']

In [545]:
nn.del_forward_pass()['layer: 2']['dW'].shape

(3, 128)

In [546]:
nn.forward_pass(data)

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      

In [551]:
nn.results['A0']

array([[ 5.69626228,  1.79499291,  8.23735869],
       [ 7.35265278,  6.02810883,  3.58616242],
       [ 1.66855587,  5.14228062,  3.91397677],
       [ 5.65037415,  5.48278021,  4.25544984],
       [ 0.89814364,  5.0109521 ,  5.77441157],
       [ 7.14154712,  3.30396138,  5.63214847],
       [ 5.42251669,  3.55184817,  3.46878872],
       [ 6.64108641,  1.20216384,  4.3881687 ],
       [ 4.62572031,  2.18575557,  6.54451518],
       [ 2.07476679,  3.96828345,  3.93470106],
       [ 3.88121598,  4.92247223,  5.78725813],
       [ 7.65191359,  5.56795074,  3.72692372],
       [ 7.36929209,  7.27418525,  5.20288485],
       [ 6.17384108,  2.41006097,  3.18356742],
       [ 3.69989098,  2.86958069,  6.2347652 ],
       [ 3.90841427,  4.36662797,  6.67352831],
       [ 4.21107696,  2.51663732,  4.58297365],
       [ 1.87132718,  4.13050497,  7.99173897],
       [ 5.85417263,  2.06052481,  4.13983191],
       [ 3.15168327,  5.65536783,  6.07725891],
       [ 4.34316957,  4.73211736,  6.357

In [552]:
nn.parameters

{'layer: 1': {'W': array([[nan, nan, nan],
         [nan, nan, nan],
         [nan, nan, nan]]),
  'b': array([1., 1., 1.]),
  'Activation': <bound method NeuralNetwork.relu of <__main__.NeuralNetwork object at 0x0000027638DBBF98>>},
 'layer: 2': {'W': array([[nan, nan],
         [nan, nan],
         [nan, nan]]),
  'b': array([1., 1.]),
  'Activation': <bound method NeuralNetwork.relu of <__main__.NeuralNetwork object at 0x0000027638DBBF98>>},
 'layer: 3': {'W': array([[nan],
         [nan]]),
  'b': array([1.]),
  'Activation': <bound method NeuralNetwork.sigmoid of <__main__.NeuralNetwork object at 0x0000027638DBBF98>>}}